In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
import requests
import re

In [3]:
url = 'https://www.kinonews.ru/top250imdb/'
page = requests.get(url)

In [4]:
page.status_code

200

In [5]:
soup = bs(page.text, 'html.parser')

In [6]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=yes">\n<link rel="icon" href="https://www.kinonews.ru/favicon.ico" type="image/x-icon">\n<link rel="shortcut icon" href="https://www.kinonews.ru/favicon.ico" type="image/x-icon">\n<meta property="twitter:domain" content="KinoNews.ru">\n<meta property="twitter:card" content="summary_large_image">\n<meta property="twitter:site" content="@KinoNewsRu">\n<meta name="twitter:creator" content="@KinoNewsRu">\n<meta name="twitter:card" content="summary_large_image">\n<title>Топ 250 лучших фильмов по версии портала о кино IMDb</title>\n<meta name=\'title\' content=\'Топ 250 лучших фильмов по версии портала о кино IMDb\'>\n<meta name=\'keywords\' content=\'топ 250 фильмов, топ 250 imdb, топ 250 лучших фильмов, топ 250 фильмов imdb, рейтинг imdb топ 250\'>\t\n<meta name=\'description\' content=\'Список 250

In [7]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, user-scalable=yes" name="viewport"/>
<link href="https://www.kinonews.ru/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="https://www.kinonews.ru/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="KinoNews.ru" property="twitter:domain"/>
<meta content="summary_large_image" property="twitter:card"/>
<meta content="@KinoNewsRu" property="twitter:site"/>
<meta content="@KinoNewsRu" name="twitter:creator"/>
<meta content="summary_large_image" name="twitter:card"/>
<title>Топ 250 лучших фильмов по версии портала о кино IMDb</title>
<meta content="Топ 250 лучших фильмов по версии портала о кино IMDb" name="title"/>
<meta content="топ 250 фильмов, топ 250 imdb, топ 250 лучших фильмов, топ 250 фильмов imdb, рейтинг imdb топ 250" name="keywords"/>
<meta content="Список 250 лучших фильмов за всю историю миров

In [8]:
result = {'title': [],'description':[] ,'rating':[],'date':[],'country':[],'genre':[],'time_film':[]}

In [9]:
pagenum = 1
for i in range(10):
    url = f'https://www.kinonews.ru/top250imdb_p{pagenum}/'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h1', class_ = 'film')
    ratings = soup.find_all('span', class_='rating-big')
    date_publish = soup.find_all('div',class_= 'bigtext')
    all_text_film = soup.find_all('div',class_='rating_rightdesc')
    descriptions = soup.find_all('div', class_='textart15')
    link = soup.find_all('a',href=True,class_= 'titlefilm')
    base_url = "https://www.kinonews.ru"
    for lin in link:
        link_url = lin['href']
        if not link_url.startswith("http"):
            link_url = base_url + link_url
        response_link = requests.get(link_url)
        soup_link = bs(response_link.text, 'html.parser')
        title = soup_link.find('h1').get_text() if soup_link.find('h1') else "Заголовок не найден"
        #description = soup_link.find('div', class_='textart15').get_text() if soup_link.find('div',class_='textart15') else "Описания нет"
        description_element = soup_link.find('div', itemprop='description')
        description = description_element.text.strip() if description_element else 'Нет описания на странице'
        rating_find = soup_link.find('span',class_='numraitview')
        rating_s = rating_find.get_text(strip=True) if rating_find else "0"
        result['title'].append(title)
        result['description'].append(description)
        result['rating'].append(rating_s)
        tds = soup_link.find_all('td')
        for i, td in enumerate(tds):
            if "Год выпуска:" in td.get_text(strip=True):
                year_td = tds[i + 1]
                year_link = year_td.find('a')
                if year_link:
                    year = year_link.text.strip()
            if "Продолжительность:" in td.get_text(strip=True):
                duration = tds[i + 1].get_text(strip=True)
            if "Жанр:" in td.get_text(strip=True):
                genre = tds[i + 1].get_text(strip=True)
            if "Страна:" in td.get_text(strip=True):
                country = tds[i + 1].get_text(strip=True)
        result['date'].append(year)
        result['time_film'].append(duration)
        result['genre'].append(genre)
        result['country'].append(country)
    pagenum += 1

In [10]:
result['title']

['Побег из Шоушенка',
 'Крестный отец',
 'Темный рыцарь',
 'Крестный отец 2',
 '12 разгневанных мужчин',
 'Список Шиндлера',
 'Властелин колец 3: Возвращение Короля',
 'Криминальное чтиво',
 'Властелин колец: Братство кольца',
 'Хороший, плохой, злой',
 'Форрест Гамп',
 'Властелин колец 2: Две крепости',
 'Бойцовский клуб',
 'Начало',
 'Звездные войны: Эпизод 5 - Империя наносит ответный удар',
 'Матрица',
 'Славные парни',
 'Пролетая над гнездом кукушки',
 'Интерстеллар',
 'Семь',
 'Эта замечательная жизнь',
 'Семь самураев',
 'Молчание ягнят',
 'Спасти рядового Райана',
 'Город Бога',
 'Жизнь прекрасна',
 'Зеленая миля',
 'Терминатор 2: Судный день',
 'Звездные войны: Эпизод 4 - Новая надежда',
 'Назад в будущее',
 'Унесенные призраками',
 'Пианист',
 'Паразиты',
 'Психо',
 'Гладиатор',
 'Король Лев',
 'Человек-паук: Паутина вселенных',
 'Отступники',
 'Одержимость',
 'Американская история Х',
 'Леон',
 'Могила светлячков',
 'Престиж',
 'Харакири',
 'Дюна: Часть вторая',
 'Подозрител

In [11]:
print("Количество нулевых значений в: ")
for i in result:
    print( i + " - " + str(result[i].count(None)))

Количество нулевых значений в: 
title - 0
description - 0
rating - 0
date - 0
country - 0
genre - 0
time_film - 0


In [12]:
file = 'films.csv'
max_length = max(len(v) for v in result.values())
for key in result:
    while len(result[key]) < max_length:
        result[key].append(None)
df = pd.DataFrame(data=result)
df.to_csv(file, index=False, encoding='utf-8')

In [13]:
df.head(60)

,title,description,rating,date,country,genre,time_film
0,Побег из Шоушенка,"""Побег из Шоушенка"" - фильм, который считается...",9.44,1994,США,"боевик,криминал,приключения",142 мин.
1,Крестный отец,"Фильм, ставший классикой своего жанра, рассказ...",9.11,1972,США,"боевик,криминал,приключения",175 мин.
2,Темный рыцарь,Бэтман вершит правосудие в Готэме. Его партнер...,9.12,2008,США,"боевик,кинокомикс,приключения,фантастика,фэнтези",152 мин.
3,Крестный отец 2,Продолжение эпохальной саги режиссера Френсиса...,8.88,1974,США,"боевик,кинокомикс,приключения,фантастика,фэнтези",202 мин.
4,12 разгневанных мужчин,Один из 12 присяжных в деле об убийстве пытае...,9.15,1957,США,"боевик,драма,семейный,спорт",96 мин.
5,Список Шиндлера,Немецкий фабрикант и член нацистской партии Ос...,9.19,1993,США,"боевик,кинокомикс,приключения,фантастика,фэнтези",195 мин.
6,Властелин колец 3: Возвращение Короля,Близится последняя битва за Средиземье. На пут...,9.48,2003,США,"боевик,драма,семейный,спорт",201 мин.
7,Криминальное чтиво,Три истории из жизни двух бандитов - Винсента ...,8.94,1994,США,"боевик,кинокомикс,приключения,фантастика,фэнтези",154 мин.
8,Властелин колец: Братство кольца,Поиски великого кольца Саурона завершены. Оно ...,9.33,2001,США,"боевик,криминал,приключения",178 мин.
9,"Хороший, плохой, злой",Действие происходит в разгар гражданской войны...,8.96,1966,США,"боевик,драма,семейный,спорт",161 мин.


In [14]:
df.describe()

,title,description,rating,date,country,genre,time_film
count,250,250,250,250,250,250,250
unique,250,250,116,87,1,3,93
top,Побег из Шоушенка,"""Побег из Шоушенка"" - фильм, который считается...",8.50,1995,США,"боевик,криминал,приключения",130 мин.
freq,1,1,8,8,250,98,8


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        250 non-null    object
 1   description  250 non-null    object
 2   rating       250 non-null    object
 3   date         250 non-null    object
 4   country      250 non-null    object
 5   genre        250 non-null    object
 6   time_film    250 non-null    object
dtypes: object(7)
memory usage: 13.8+ KB


In [16]:
janr = ['драма', 'драма, фэнтези, криминал','драма, комедия, мелодрама, история, военный', 'драма, биография, история, военный','драма, комедия','фантастика, боевик, триллер, драма, детектив', 'боевик, триллер, драма, криминал',
        'мультфильм, мюзикл, драма, приключения, семейный','триллер, драма, криминал','комедия, фантастика, приключения','военный, комедия, драма, мелодрама',
        'драма, комедия, криминал','драма, криминал','криминал, драма','комедия, мелодрама, криминал','триллер, фантастика, драма, детектив',
        'биография, драма, мелодрама','фантастика, драма, приключения','фэнтези, приключения, драма, боевик','история, боевик, драма','фантастика, комедия, приключения', 
        'боевик, комедия, криминал','фантастика, боевик','комедия, криминал','криминал, драма, триллер','криминал, биография, комедия','драма, военный, биография, музыка',
        'фэнтези, приключения, драма, боевик','военный, драма, комедия','мультфильм, фантастика, приключения, семейный','мультфильм, фэнтези, комедия, приключения, семейный, музыка',
        'криминал, комедия, боевик','фэнтези, приключения, драма, боевик','драма, криминал','комедия, драма, криминал, детектив',
        'фэнтези, боевик, приключения','триллер, детектив, драма','фантастика, боевик, триллер, криминал, драма','драма','мелодрама, история, триллер, драма',
        'драма, детектив, криминал','драма','драма, фантастика','мелодрама, комедия, криминал, музыка','драма, семейный, биография',
        'комедия, мелодрама, драма','аниме, мультфильм, фэнтези, приключения, семейный','комедия, приключения, мелодрама, мюзикл','драма, военный, история',
        'фэнтези, драма, мелодрама, семейный']

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        250 non-null    object
 1   description  250 non-null    object
 2   rating       250 non-null    object
 3   date         250 non-null    object
 4   country      250 non-null    object
 5   genre        250 non-null    object
 6   time_film    250 non-null    object
dtypes: object(7)
memory usage: 13.8+ KB
